# Acquiring and Processing Information on the World's Largest Banks

# Project Scenario:

You have been hired as a data engineer by research organization. Your boss has asked you to create a code that can be used to compile the list of the top 10 largest banks in the world ranked by market capitalization in billion USD. Further, the data needs to be transformed and stored in GBP, EUR and INR as well, in accordance with the exchange rate information that has been made available to you as a CSV file. The processed information table is to be saved locally in a CSV format and as a database table.

Your job is to create an automated system to generate this information so that the same can be executed in every financial quarter to prepare the report.

Particulars of the code to be made have been shared below.

Code name: banks_project.py

Data URL

Exchange rate CSV path

# Initial setup

Before you start building the code, you need to install the required libraries for it.

The libraries needed for the code are as follows:

1. requests - The library used for accessing the information from the URL.

2. bs4 - The library containing the BeautifulSoup function used for webscraping.

3. pandas - The library used for processing the extracted data, storing it to required formats and communicating with the databases.

4. sqlite3 - The library required to create a database server connection.

5. numpy - The library required for the mathematical rounding operation as required in the objectives.

6. datetime - The library containing the function datetime used for extracting the timestamp for logging purposes.

While requests, sqlite3, and datetime come bundled with python, the other libraries will have to be installed.

In [14]:
!pip install numpy pandas bs4 wget

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9680 sha256=5fad042e430f18918ad0d16d7cc6565480a77edb31dc4f0bd358e55af33c10df
  Stored in directory: c:\users\krist\appdata\local\pip\cache\wheels\01\46\3b\e29ffbe4ebe614ff224bad40fc6a5773a67a163251585a13a9
Successfully built wget


Download the required exchange rate file using the terminal command:

# Code structure

In [17]:
# Importing the required libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime 

# Suppress generated warnings
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

## Task 1: Logging function

Write the function to log the progress of the code, log_progress(). The function accepts the message to be logged and enters it to a text file code_log.txt.

The format to be used for logging must have the syntax:

Logging needs to be done using the log_progress() funciton. This function will be called multiple times throughout the execution of this code and will be asked to add a log entry in a .txt file, etl_project_log.txt. The entry is supposed to be in the following format:

Here, message text is passed to the function as an argument. Each entry must be in a separate line.

In [22]:
def log_progress(message):
    ''' This function logs the mentioned message at a given stage of the 
    code execution to a log file. Function returns nothing.'''
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second 
    now = datetime.now() # get current timestamp 
    timestamp = now.strftime(timestamp_format) 
    with open("./code_log.txt","a") as f: 
        f.write(timestamp + ' : ' + message + '\n')    

## Task 2 : Extraction of data

Download the required exchange rate file

In [25]:
# Download the CSV data first into a local `exchange_rate.csv` file
import wget
wget.download("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv")

'exchange_rate (1).csv'

Analyze the webpage on the given URL:

Identify the position of the required table under the heading By market capitalization. Write the function extract() to retrieve the information of the table to a Pandas data frame.

Note: Remember to remove the last character from the Market Cap column contents, like, '\n', and typecast the value to float format.

Write a function call for extract() and print the returning data frame.

Make the relevant log entry.

In [28]:
def extract(url, table_attribs):
    ''' The purpose of this function is to extract the required
    information from the website and save it to a dataframe. The
    function returns the dataframe for further processing. '''

    # Extract the web page as text
    page = requests.get(url).text

    # Parse the text into an HTML object
    data = BeautifulSoup(page,'html.parser')

    # Create an empty pandas DataFrame named df with columns as the table_attribs.
    df = pd.DataFrame(columns=table_attribs)

    # Extract all 'tbody' attributes of the HTML object and then extract all the rows of the index 2 table using the 'tr' attribute.
    tables = data.find_all('tbody')
    rows = tables[0].find_all('tr')

    # Check the contents of each row, having attribute ‘td’
    for row in rows:
        col = row.find_all('td')
        if len(col)!=0:
            data_dict = {"Name": col[1].find_all("a")[1]["title"],
                         "MC_USD_Billion": float(col[2].contents[0][:-1])}
            df1 = pd.DataFrame(data_dict, index=[0])

            # Append all these dictionaries one by one to the dataframe.
            df = pd.concat([df,df1], ignore_index=True)
    return df

## Task 3 : Transformation of data

The Transform function needs to perform the following tasks:

1. Read the exchange rate CSV file and convert the contents to a dictionary so that the contents of the first columns are the keys to the dictionary and the contents of the second column are the corresponding values.

2. Add 3 different columns to the dataframe, viz. MC_GBP_Billion, MC_EUR_Billion and MC_INR_Billion, each containing the content of MC_USD_Billion scaled by the corresponding exchange rate factor. Remember to round the resulting data to 2 decimal places.

A sample statement is being provided for adding the MC_GBP_Billion column. You can use this to add the other two statements on your own.

In [33]:
def transform(df):
    ''' This function accesses the CSV file for exchange rate
    information, and adds three columns to the data frame, each
    containing the transformed version of Market Cap column to
    respective currencies'''

    # Read exchange rate CSV file
    df_exchange_rate = pd.read_csv('./exchange_rate.csv')

    # Convert to dict
    exchange_rate = df_exchange_rate.set_index('Currency').to_dict()['Rate']

    # Add MC_GBP_Billion, MC_EUR_Billion, and MC_INR_Billion
    # columns to dataframe. Round off to two decimals
    df["MC_GBP_Billion"] = [np.round(x * exchange_rate["GBP"], 2) for x in df["MC_USD_Billion"]]
    df["MC_EUR_Billion"] = [np.round(x * exchange_rate["EUR"], 2) for x in df["MC_USD_Billion"]]
    df["MC_INR_Billion"] = [np.round(x * exchange_rate["INR"], 2) for x in df["MC_USD_Billion"]]
    
    return df

## Task 4: Loading to CSV

Write the function to load the transformed data frame to a CSV file, like load_to_csv(), in the path mentioned in the project scenario.

In [36]:
def load_to_csv(df, csv_path):
    ''' This function saves the final dataframe as a `CSV` file 
    in the provided path. Function returns nothing.'''
    df.to_csv(csv_path)


## Task 5: Loading to Database

Write the function to load the transformed data frame to an SQL database, like, load_to_db(). Use the database and table names as mentioned in the project scenario.

Before calling this function, initiate the connection to the SQLite3 database server with the name Banks.db. Pass this connection object, along with the required table name Largest_banks and the transformed data frame, to the load_to_db() function in the function call.

In [39]:
def load_to_db(df, sql_connection, table_name):
    ''' This function saves the final dataframe to as a database table
    with the provided name. Function returns nothing.'''
    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)


## Task 6: Function to Run queries on Database

Write the function run_queries() that accepts the query statement, and the SQLite3 Connection object, and generates the output of the query. The query statement should be printed along with the query output.

In [42]:
def run_query(query_statement, sql_connection):
    ''' This function runs the stated query on the database table and
    prints the output on the terminal. Function returns nothing. '''
    print(query_statement)
    query_output = pd.read_sql(query_statement, sql_connection)
    print(query_output)

## Define required entities

In [44]:
url = 'https://web.archive.org/web/20230908091635 /https://en.wikipedia.org/wiki/List_of_largest_banks'
table_attribs = ["Name", "MC_USD_Billion"]
db_name = 'Banks.db'
table_name = 'Largest_banks'
csv_path = './Largest_banks_data.csv'
log_file = "./code_log.txt"

## Function calls

Declaring known values

In [47]:
log_progress('Preliminaries complete. Initiating ETL process')

Call extract() function

In [49]:
df = extract(url, table_attribs)
print(df)

log_progress('Data extraction complete. Initiating Transformation process')

                                      Name  MC_USD_Billion
0                           JPMorgan Chase          432.92
1                          Bank of America          231.52
2  Industrial and Commercial Bank of China          194.56
3               Agricultural Bank of China          160.68
4                                HDFC Bank          157.91
5                              Wells Fargo          155.87
6                                     HSBC          148.90
7                           Morgan Stanley          140.83
8                  China Construction Bank          139.82
9                            Bank of China          136.81


Call transform() function

In [51]:
df = transform(df)
print(df)

log_progress('Data transformation complete. Initiating loading process')

                                      Name  MC_USD_Billion  MC_GBP_Billion  \
0                           JPMorgan Chase          432.92          346.34   
1                          Bank of America          231.52          185.22   
2  Industrial and Commercial Bank of China          194.56          155.65   
3               Agricultural Bank of China          160.68          128.54   
4                                HDFC Bank          157.91          126.33   
5                              Wells Fargo          155.87          124.70   
6                                     HSBC          148.90          119.12   
7                           Morgan Stanley          140.83          112.66   
8                  China Construction Bank          139.82          111.86   
9                            Bank of China          136.81          109.45   

   MC_EUR_Billion  MC_INR_Billion  
0          402.62        35910.71  
1          215.31        19204.58  
2          180.94        16138.75

Call load_to_csv()

In [53]:
load_to_csv(df, csv_path)

log_progress('Data saved to CSV file')

Initiate SQLite3 connection

In [55]:
sql_connection = sqlite3.connect(db_name)

log_progress('SQL Connection initiated.')

Call load_to_db()

In [57]:
load_to_db(df, sql_connection, table_name)

log_progress('Data loaded to Database as a table, Executing queries')

Call run_query()

1. Print the contents of the entire table

In [60]:
query_statement = f"SELECT * from {table_name}"
run_query(query_statement, sql_connection)

SELECT * from Largest_banks
                                      Name  MC_USD_Billion  MC_GBP_Billion  \
0                           JPMorgan Chase          432.92          346.34   
1                          Bank of America          231.52          185.22   
2  Industrial and Commercial Bank of China          194.56          155.65   
3               Agricultural Bank of China          160.68          128.54   
4                                HDFC Bank          157.91          126.33   
5                              Wells Fargo          155.87          124.70   
6                                     HSBC          148.90          119.12   
7                           Morgan Stanley          140.83          112.66   
8                  China Construction Bank          139.82          111.86   
9                            Bank of China          136.81          109.45   

   MC_EUR_Billion  MC_INR_Billion  
0          402.62        35910.71  
1          215.31        19204.58  
2    

2. Print the average market capitalization of all the banks in Billion USD.

In [62]:
query_statement = f"SELECT AVG(MC_GBP_Billion) FROM {table_name}"
run_query(query_statement, sql_connection)

SELECT AVG(MC_GBP_Billion) FROM Largest_banks
   AVG(MC_GBP_Billion)
0              151.987


3. Print only the names of the top 5 banks

In [64]:
query_statement = f"SELECT Name from {table_name} LIMIT 5"
run_query(query_statement, sql_connection)

SELECT Name from Largest_banks LIMIT 5
                                      Name
0                           JPMorgan Chase
1                          Bank of America
2  Industrial and Commercial Bank of China
3               Agricultural Bank of China
4                                HDFC Bank


End process

In [66]:
log_progress('Process Complete.')

Close SQLite3 connection

In [68]:
sql_connection.close()
log_progress('Server Connection closed')

## Task 7: Verify log entries

After updating all the log_progress() function calls, you have to run the code for a final execution. However, you will first have to remove the code_log.txt file, that would have been created and updated throughout the multiple executions of the code in this lab. You may remove the file using the following command on a terminal.

In [71]:
with open(log_file, "r") as log:
    LogContent = log.read()
    print(LogContent)

2025-May-16-13:12:36 : Preliminaries complete. Initiating ETL process
2025-May-16-13:12:39 : Data extraction complete. Initiating Transformation process
2025-May-16-13:12:39 : Data transformation complete. Initiating loading process
2025-May-16-13:12:40 : Data saved to CSV file
2025-May-16-13:12:40 : SQL Connection initiated.
2025-May-16-13:12:40 : Data loaded to Database as a table, Executing queries
2025-May-16-13:12:40 : Process Complete.
2025-May-16-13:12:40 : Server Connection closed

